スペクトラム補償付きWS制御

In [ ]:
import vxi11
from WSMethods import uploadProfile, getWSrange
from convinience import Spectrum, Band, synthesisSpectrum
from convinience import gaussian_profile, compensation, calc_totalPower ,attenuation
import numpy as np
import ipywidgets as widgets
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display
from ipyfilechooser import FileChooser
import plotly.io as pio
pio.renderers.default='notebook' # for vscode ,maybe 'colab' on jupyterlab 

In [ ]:
class wrapper:
    def __init__(self, instrument: vxi11.Instrument):
        self.instr = instrument

    def __enter__(self):
        return self.instr

    def __exit__(self, ex_type, ex_value, trace):
        self.instr.close()


def setOSArange():
    global osa_ip
    global startFreq
    global endFreq
    """startFreq < endFreq
    """
    assert startFreq <= endFreq
    with wrapper(vxi11.Instrument(osa_ip)) as instr:
        instr.write(f':SENSe:WAVelength:STARt {float(endFreq)}THZ') # short wavelength
        instr.write(f':SENSe:WAVelength:STOP {float(startFreq)}THZ') # long wavelength

def getOSA():
    global osa_ip
    global trace_name
    global optWavelength
    global optFreq
    global optSpectrum
    c = 299792458

    with wrapper(vxi11.Instrument(osa_ip)) as instr:# 真空の光速
        instr.write(':SENSE:CORRECTION:RVELOCITY:MEDIUM VACUUM')

    with wrapper(vxi11.Instrument(osa_ip)) as instr:
        instr.write(':UNIT:X WAVelength')
    # 基本的にどのように設定しようが、波長単位でデータが帰ってくるので関係なし
    

    #光スペアナのx軸（波長）取得
    with wrapper(vxi11.Instrument(osa_ip) ) as instr:
        wavelength = instr.ask(':TRACe:X? '+trace_name)
        wavelength = wavelength.split(',')
        wavelength = [float(ii) for ii in wavelength]
        wavelength = np.array(wavelength)# m

    freq = c/wavelength/1E12 # THz

    #光スペアナのy軸（dBm/nm）取得
    with wrapper(vxi11.Instrument(osa_ip)) as instr:
        pnbw = instr.ask(':CALCULATE:MATH:TRF:PNBW:BAND?')#正規化帯域幅

    with wrapper(vxi11.Instrument(osa_ip)) as instr:
        data = instr.ask(':TRACe:Y:PDENsity? '+trace_name+',0.1nm')
        data = data.split(',')
        data = [float(ii) for ii in data]
        data = np.array(data)

    optWavelength = wavelength
    optFreq = freq
    optSpectrum = data
    

def get_wsrange():
    global startFreq, endFreq
    global ws_ip
    startFreq, endFreq = getWSrange(ws_ip)
    print(f'startFreq = {startFreq}')
    print(f'  endFreq = {endFreq}')

def plot_spectrum_freq():
    global optWavelength
    global optFreq
    global optSpectrum
    fig = make_subplots(rows=1, cols=2, subplot_titles=("power", "phase"))
    go_element = lambda x,y,name: go.Scatter(x=x,y=y,mode='lines',name=name)
    fig.add_trace(go_element(optFreq, optSpectrum, 'optSpectrum'), row=1, col=1)
    fig.update_xaxes(title_text="Freq", row=1, col=1)
    fig.update_yaxes(title_text="power", row=1, col=1)
    fig.update_xaxes(title_text="Freq", row=1, col=2)
    fig.update_yaxes(title_text="Phase", row=1, col=2)
    fig.show()

def plot_spectrum_wavelength():
    global optWavelength
    global optFreq
    global optSpectrum
    fig = make_subplots(rows=1, cols=2, subplot_titles=("power", "phase"))
    go_element = lambda x,y,name: go.Scatter(x=x,y=y,mode='lines',name=name)
    fig.add_trace(go_element(optWavelength, optSpectrum, 'optSpectrum'), row=1, col=1)
    fig.update_xaxes(title_text="wavelength", row=1, col=1)
    fig.update_yaxes(title_text="power", row=1, col=1)
    fig.update_xaxes(title_text="wavelength", row=1, col=2)
    fig.update_yaxes(title_text="Phase", row=1, col=2)
    fig.show()

def save_spectrum_data():
    global optSpectrum
    global optFreq
    global comment
    filename_optFreq = f'optFreq_{comment}.npy'
    filename_optSpectrum = f'optSpectrum_{comment}.npy'
    np.save(filename_optFreq, optFreq)
    np.save(filename_optSpectrum, optSpectrum)


def gaussian_gen():
    global centerWavelength
    global bandWidth
    global dispersion
    global filterSpectrum
    startFreq, endFreq = 191.05, 196.475
    ws_Freq_resolution = 0.001
    band = Band()
    band.frequency          = np.arange(startFreq, endFreq, ws_Freq_resolution)
    filterSpectrum          = gaussian_profile(
                centerWavelength    = centerWavelength,
                bandWidth           = bandWidth, band=band,
                dispersion          = dispersion
            )

    fig = make_subplots(rows=1, cols=2, subplot_titles=("power", "phase"))
    go_element = lambda x,y,name: go.Scatter(x=x,y=y,mode='lines',name=name)
    fig.add_trace(go_element(filterSpectrum.wavelength, filterSpectrum.powerdensity, 'filterSpectrum'), row=1, col=1)
    fig.add_trace(go_element(filterSpectrum.wavelength, filterSpectrum.phase, 'filterSpectrum'), row=1, col=2)
    fig.update_xaxes(title_text="wavelength", row=1, col=1)
    fig.update_yaxes(title_text="power", row=1, col=1)
    fig.update_xaxes(title_text="wavelength", row=1, col=2)
    fig.update_yaxes(title_text="Phase", row=1, col=2)
    fig.show()

def compensate():
    global filterSpectrum
    global attenuation_db
    global compensated_Spectrum
    optSpectrumTemp                = Spectrum()
    optSpectrumTemp.band           = Band()
    optSpectrumTemp.frequency      = optFreq
    optSpectrumTemp.powerdensity   = optSpectrum
    optSpectrumTemp.phase          = np.zeros_like(optSpectrumTemp.frequency)

    startFreq, endFreq = 191.05, 196.475
    ws_Freq_resolution = 0.001
    band = Band()
    band.frequency          = np.arange(startFreq, endFreq, ws_Freq_resolution)

    compensated_Spectrum = compensation(
            optSpectrumTemp,
            targetSpectrum      =filterSpectrum,
            band                =band
        )
    compensated_Spectrum = attenuation(compensated_Spectrum, attenuation_db)

    fig = make_subplots(rows=1, cols=2, subplot_titles=("power", "phase"))
    go_element = lambda x,y,name: go.Scatter(x=x,y=y,mode='lines',name=name)
    fig.add_trace(go_element(filterSpectrum.wavelength, filterSpectrum.powerdensity, 'filterSpectrum'), row=1, col=1)
    fig.add_trace(go_element(filterSpectrum.wavelength, filterSpectrum.phase, 'filterSpectrum'), row=1, col=2)
    fig.add_trace(go_element(compensated_Spectrum.wavelength, compensated_Spectrum.powerdensity, 'compensated_Spectrum'), row=1, col=1)
    fig.add_trace(go_element(compensated_Spectrum.wavelength, compensated_Spectrum.phase, 'compensated_Spectrum'), row=1, col=2)
    fig.update_xaxes(title_text="wavelength", row=1, col=1)
    fig.update_yaxes(title_text="power", row=1, col=1)
    fig.update_xaxes(title_text="wavelength", row=1, col=2)
    fig.update_yaxes(title_text="Phase", row=1, col=2)
    fig.show()



def upload_spectrum():
    global ws_ip
    filterPort = np.ones(compensated_Spectrum.powerdensity.shape)
    r = uploadProfile(ws_ip, compensated_Spectrum.frequency, -(compensated_Spectrum.powerdensity), compensated_Spectrum.phase, filterPort)


In [ ]:
def simple_ui():
    button_clear_output = widgets.Button(description='表示クリア')
    button_get_wsrange = widgets.Button(description='WSレンジ読取')
    button_setOSArange = widgets.Button(description='OSAレンジ設定')
    button_getOSA = widgets.Button(description='OSA読込')
    button_plot_spectrum_freq = widgets.Button(description='周波数プロット')
    button_plot_spectrum_wavelength = widgets.Button(description='波長プロット')
    button_save_spectrum_data = widgets.Button(description='スペクトラム保存')
    button_gaussian_gen = widgets.Button(description='ガウシアンスペクトラム')
    button_compensate = widgets.Button(description='補償')
    button_upload_spectrum = widgets.Button(description='アップロード')
    filechooser = FileChooser('./')
    button_load_as_optSpectrum = widgets.Button(description='spectrum.npy読込')
    button_load_as_optFreq = widgets.Button(description='freq.npy読込')
    button_load_as_optWavelength = widgets.Button(description='wavelength.npy読込')
    button_input_field = widgets.Button(description='変数反映')

    text_comment = widgets.Text(value='',placeholder='文字を入力',description='comment',disabled=False)
    text_ws_ip = widgets.Text(value='169.254.6.8',placeholder='ws_ip',description='ws_ip',disabled=False)
    text_osa_ip = widgets.Text(value='169.254.6.9',placeholder='osa_ip',description='os_ip',disabled=False)
    text_trace_name = widgets.Text(value='TRA',placeholder='trace_name',description='trace_name',disabled=False)
    float_centerWavelength = widgets.FloatText(value=1545.3, placeholder='nm',description='centerWavelength',disabled=False)
    float_bandWidth = widgets.FloatText(value=1, placeholder='nm', description='bandWidth',disabled=False)
    float_dispersion = widgets.FloatText(value=0, placeholder='ps/nm', description='dispersion', disabled=False)
    float_attenuation_db = widgets.FloatText(value=0, placeholder='dB', description='attenuation_db', disabled=False)

    output = widgets.Output(layour={'border': '1px solid black'})
    def wrapped_func_factory(func):
        def new_func(ui_element):
            with output:
                print(f"exec func {func.__name__}")
                func()
                print(f"complete {func.__name__}")
        return new_func
    button_clear_output.on_click(lambda button: output.clear_output(wait=False))
    button_get_wsrange.on_click(wrapped_func_factory(get_wsrange))
    button_setOSArange.on_click(wrapped_func_factory(setOSArange))
    button_getOSA.on_click(wrapped_func_factory(getOSA))
    button_plot_spectrum_freq.on_click(wrapped_func_factory(plot_spectrum_freq))
    button_plot_spectrum_wavelength.on_click(wrapped_func_factory(plot_spectrum_wavelength))
    button_save_spectrum_data.on_click(wrapped_func_factory(save_spectrum_data))
    button_gaussian_gen.on_click(wrapped_func_factory(gaussian_gen))
    button_compensate.on_click(wrapped_func_factory(compensate))
    button_upload_spectrum.on_click(wrapped_func_factory(upload_spectrum))

    def load_npy_factory(variable_name):
        def load_npy():
            c = 299792458
            choosed_file_path = filechooser.selected
            global optSpectrum, optFreq, optWavelength
            if variable_name == 'spectrum':
                optSpectrum = np.load(choosed_file_path)
                print(f'optSpectrum.shape={optSpectrum.shape}')
            elif variable_name == 'freq':
                optFreq = np.load(choosed_file_path)
                optWavelength = c/optFreq/1E12
                print(f'optFreq.shape={optFreq.shape}')
            elif variable_name == 'wavelength':
                optWavelength = np.load(choosed_file_path)
                optFreq = c/optWavelength/1E12
                print(f'optWavelength.shape={optWavelength.shape}')
        return load_npy
    
    button_load_as_optSpectrum.on_click(wrapped_func_factory(load_npy_factory('spectrum')))
    button_load_as_optFreq.on_click(wrapped_func_factory(load_npy_factory('freq')))
    button_load_as_optWavelength.on_click(wrapped_func_factory(load_npy_factory('wavelength')))    
    
    def load_input_field():
        global comment
        global ws_ip
        global osa_ip
        global trace_name
        global centerWavelength
        global bandWidth
        global dispersion
        global attenuation_db
        comment = text_comment.value
        ws_ip = text_ws_ip.value
        osa_ip = text_osa_ip.value
        trace_name = text_trace_name.value
        centerWavelength = float_centerWavelength.value
        bandWidth = float_bandWidth.value
        dispersion = float_dispersion.value
        attenuation_db = float_attenuation_db.value
    button_input_field.on_click(lambda button: load_input_field())
    

    display(
        widgets.VBox([
            widgets.HBox([button_clear_output,button_get_wsrange, button_setOSArange, button_getOSA, button_plot_spectrum_freq, button_plot_spectrum_wavelength, button_save_spectrum_data]),
            widgets.HBox([button_gaussian_gen, button_compensate, button_upload_spectrum]),
            widgets.HBox([filechooser, button_load_as_optSpectrum, button_load_as_optFreq, button_load_as_optWavelength]),
            widgets.HBox([text_comment, text_ws_ip, text_osa_ip, text_trace_name, button_input_field]),
            widgets.HBox([float_centerWavelength,float_bandWidth,float_dispersion,float_attenuation_db]),
            output,
        ])
    )
    

In [ ]:
simple_ui()